# Plugin Selector

> Reusable plugin selection UI components with multiple display strategies

In [ ]:
#| default_exp components.selector

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import List, Dict, Any, Optional, Callable
from fasthtml.common import *
from cjm_fasthtml_daisyui.components.actions.button import btn, btn_colors
from cjm_fasthtml_daisyui.components.data_input.select import select as dui_select
from cjm_fasthtml_daisyui.components.data_display.card import card, card_body, card_title
from cjm_fasthtml_daisyui.components.data_display.badge import badge, badge_colors, badge_sizes
from cjm_fasthtml_daisyui.components.data_display.table import table, table_modifiers
from cjm_fasthtml_daisyui.components.actions.button import btn, btn_sizes, btn_styles
from cjm_fasthtml_daisyui.utilities.semantic_colors import bg_dui, text_dui, border_dui
from cjm_fasthtml_tailwind.utilities.spacing import p, m
from cjm_fasthtml_tailwind.utilities.sizing import w
from cjm_fasthtml_tailwind.utilities.typography import font_size, font_weight
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import flex_display, gap, items, justify
from cjm_fasthtml_tailwind.utilities.borders import border
from cjm_fasthtml_tailwind.core.base import combine_classes

from cjm_fasthtml_plugins.core.metadata import PluginMetadata
from cjm_fasthtml_plugins.core.execution_mode import PluginExecutionMode
from cjm_fasthtml_plugins.components.html_ids import PluginSelectorHtmlIds as HtmlIds

## Metadata Display Helpers

Functions to format plugin metadata for display.

In [ ]:
#| export
def format_execution_mode(
    mode: PluginExecutionMode  # Execution mode enum
) -> str:  # Human-readable string
    """Format execution mode for display."""
    mode_labels = {
        PluginExecutionMode.IN_PROCESS: "Local (In-Process)",
        PluginExecutionMode.SUBPROCESS: "Local (Subprocess)",
        PluginExecutionMode.DOCKER: "Local (Docker)",
        PluginExecutionMode.CONDA_ENV: "Local (Conda)",
        PluginExecutionMode.EXTERNAL_SERVICE: "Local (External Service)",
        PluginExecutionMode.CLOUD_GPU: "Cloud GPU",
        PluginExecutionMode.CLOUD_CPU: "Cloud CPU",
    }
    return mode_labels.get(mode, mode.value)

In [ ]:
#| export
def get_execution_badge_color(
    mode: PluginExecutionMode  # Execution mode enum
) -> str:  # DaisyUI badge color class
    """Get badge color for execution mode."""
    if mode in [PluginExecutionMode.IN_PROCESS, PluginExecutionMode.SUBPROCESS,
                PluginExecutionMode.DOCKER, PluginExecutionMode.CONDA_ENV,
                PluginExecutionMode.EXTERNAL_SERVICE]:
        return str(badge_colors.success)
    elif mode in [PluginExecutionMode.CLOUD_GPU, PluginExecutionMode.CLOUD_CPU]:
        return str(badge_colors.info)
    else:
        return str(badge_colors.neutral)

In [ ]:
#| export
def extract_plugin_features(
    plugin: PluginMetadata,  # Plugin metadata
    config: Optional[Dict[str, Any]] = None  # Optional plugin configuration
) -> List[tuple]:  # List of (label, value, badge_color) tuples
    """Extract key features from plugin metadata for display."""
    features = []
    
    # Execution mode
    features.append((
        "Execution",
        format_execution_mode(plugin.execution_mode),
        get_execution_badge_color(plugin.execution_mode)
    ))
    
    # Check for streaming support in config schema
    if "supports_streaming" in plugin.config_schema.get("properties", {}):
        supports_streaming = config.get("supports_streaming", False) if config else False
        features.append((
            "Streaming",
            "Supported" if supports_streaming else "Not Supported",
            str(badge_colors.info) if supports_streaming else str(badge_colors.neutral)
        ))
    
    # Cloud-specific info
    if plugin.is_cloud_execution() and plugin.remote_resource:
        if plugin.remote_resource.estimated_cost_per_hour:
            features.append((
                "Cost",
                f"${plugin.remote_resource.estimated_cost_per_hour:.2f}/hr",
                str(badge_colors.warning)
            ))
        if plugin.remote_resource.gpu_type:
            features.append((
                "GPU",
                f"{plugin.remote_resource.gpu_count}x {plugin.remote_resource.gpu_type}",
                str(badge_colors.primary)
            ))
    
    # Local resource info from config
    if config and plugin.is_local_execution():
        if "device" in config:
            device = config["device"]
            features.append((
                "Device",
                device.upper(),
                str(badge_colors.primary) if device == "cuda" else str(badge_colors.neutral)
            ))
        
        # Memory requirements
        if "memory_gb" in config:
            features.append((
                "Memory",
                f"{config['memory_gb']}GB",
                str(badge_colors.neutral)
            ))
    
    return features

## Plugin Details Display

Show detailed information about the selected plugin.

In [ ]:
#| export
def render_plugin_details(
    plugin: PluginMetadata,  # Plugin to display
    config: Optional[Dict[str, Any]] = None  # Optional plugin configuration
) -> FT:  # Card element with plugin details
    """Render detailed information about a plugin."""
    features = extract_plugin_features(plugin, config)
    
    return Div(
        Div(
            # Title and description
            H3(plugin.title, cls=str(card_title)),
            P(
                plugin.description or "No description available.",
                cls=combine_classes(
                    text_dui.base_content.opacity(80),
                    font_size.sm,
                    m.b(4)
                )
            ),
            
            # Feature badges
            Div(
                *[
                    Div(
                        Span(
                            f"{label}:",
                            cls=combine_classes(
                                font_weight.medium,
                                text_dui.base_content.opacity(70),
                                font_size.sm
                            )
                        ),
                        Span(
                            value,
                            cls=combine_classes(
                                badge,
                                badge_color,
                                badge_sizes.sm,
                                m.l(2)
                            )
                        ),
                        cls=combine_classes(
                            flex_display,
                            items.center,
                            m.b(2)
                        )
                    )
                    for label, value, badge_color in features
                ],
                cls=str(m.b(4))
            ),
            
            # Version info
            P(
                f"Version: {plugin.version or 'Unknown'}",
                cls=combine_classes(
                    text_dui.base_content.opacity(60),
                    font_size.xs
                )
            ),
            
            cls=str(card_body)
        ),
        id=HtmlIds.PLUGIN_DETAILS,
        cls=combine_classes(
            card,
            bg_dui.base_200,
            m.t(4)
        )
    )

## Dropdown Selector Strategy

The primary UI pattern: dropdown selection with details panel.

In [ ]:
#| export
def render_dropdown_selector(
    plugins: List[PluginMetadata],  # Available plugins
    selected_plugin_id: Optional[str] = None,  # Currently selected plugin ID
    selection_endpoint: str = None,  # HTMX endpoint for confirming selection
    details_endpoint: Optional[str] = None,  # HTMX endpoint for updating details (optional)
    target_id: str = None,  # HTMX target element ID for final selection
    label: str = "Select Plugin",  # Label for the dropdown
    confirm_button_text: str = "Continue with Selected Plugin",  # Text for confirmation button
    get_plugin_config: Optional[Callable[[str], Dict[str, Any]]] = None,  # Function to get plugin config
    show_comparison_toggle: bool = True  # Whether to show comparison table toggle
) -> FT:  # Complete plugin selector UI
    """Render plugin selector with dropdown and details panel.
    
    The dropdown updates the details panel for browsing, while an explicit
    button confirms the selection and proceeds to the next step.
    """
    
    if not plugins:
        return P(
            "No plugins available.",
            cls=str(text_dui.base_content.opacity(60))
        )
    
    # Find selected plugin
    selected_plugin = None
    selected_config = None
    if selected_plugin_id:
        selected_plugin = next(
            (p for p in plugins if p.get_unique_id() == selected_plugin_id),
            None
        )
        if selected_plugin and get_plugin_config:
            selected_config = get_plugin_config(selected_plugin_id)
    
    # If no selection, default to first plugin
    if not selected_plugin and plugins:
        selected_plugin = plugins[0]
        selected_plugin_id = selected_plugin.get_unique_id()
        if get_plugin_config:
            selected_config = get_plugin_config(selected_plugin.get_unique_id())
    
    # Build dropdown attributes for updating details only
    select_attrs = {
        "name": "plugin_id",
        "id": HtmlIds.PLUGIN_SELECT,
        "cls": combine_classes(dui_select, w.full)
    }
    
    # If details_endpoint provided, use it to update details on change
    if details_endpoint:
        select_attrs.update({
            "hx_get": details_endpoint,
            "hx_target": HtmlIds.as_selector(HtmlIds.PLUGIN_DETAILS),
            "hx_swap": "outerHTML",
            "hx_trigger": "change",
            "hx_include": f"#{HtmlIds.PLUGIN_SELECT}"
        })
    
    return Div(
        # Dropdown section
        Div(
            Label(
                label,
                cls=combine_classes(
                    font_weight.semibold,
                    m.b(2)
                )
            ),
            Select(
                *[
                    Option(
                        plugin.title,
                        value=plugin.get_unique_id(),
                        selected=(plugin.get_unique_id() == selected_plugin_id)
                    )
                    for plugin in plugins
                ],
                **select_attrs
            ),
            cls=str(m.b(4))
        ),
        
        # Details panel
        render_plugin_details(selected_plugin, selected_config) if selected_plugin else None,
        
        # Confirmation button
        Div(
            Button(
                confirm_button_text,
                hx_post=selection_endpoint if selection_endpoint else "#",
                hx_target=f"#{target_id}" if target_id else None,
                hx_swap="innerHTML",
                hx_include=f"#{HtmlIds.PLUGIN_SELECT}",
                cls=combine_classes(
                    btn,
                    btn_colors.primary,
                    btn_sizes.md,
                    m.t(4)
                )
            ),
            cls=str(m.t(4))
        ) if selection_endpoint and target_id else None,
        
        # Optional comparison toggle
        Div(
            Button(
                "Compare All Plugins",
                hx_get=selection_endpoint.replace("select", "compare") if selection_endpoint else "#",
                hx_target=HtmlIds.as_selector(HtmlIds.PLUGIN_COMPARISON),
                hx_swap="innerHTML",
                cls=combine_classes(
                    btn,
                    btn_styles.outline,
                    btn_sizes.sm,
                    m.t(2)
                )
            ),
            cls=str(m.t(2))
        ) if show_comparison_toggle and selection_endpoint else None,
        
        # Comparison container
        Div(id=HtmlIds.PLUGIN_COMPARISON, cls=str(m.t(4))) if show_comparison_toggle else None,
        
        id=HtmlIds.PLUGIN_SELECTOR
    )

## Comparison Table Strategy

Show all plugins in a comparison table.

In [ ]:
#| export
def render_comparison_table(
    plugins: List[PluginMetadata],  # Plugins to compare
    selected_plugin_id: Optional[str] = None,  # Currently selected plugin ID
    get_plugin_config: Optional[Callable[[str], Dict[str, Any]]] = None  # Function to get plugin config
) -> FT:  # Table element with plugin comparison
    """Render a comparison table of all plugins."""
    
    if not plugins:
        return P(
            "No plugins to compare.",
            cls=str(text_dui.base_content.opacity(60))
        )
    
    # Extract all unique feature keys
    all_features = {}
    for plugin in plugins:
        config = get_plugin_config(plugin.get_unique_id()) if get_plugin_config else None
        features = extract_plugin_features(plugin, config)
        for label, value, _ in features:
            if label not in all_features:
                all_features[label] = []
            all_features[label].append((plugin.get_unique_id(), value))
    
    return Div(
        Div(
            H3(
                "Plugin Comparison",
                cls=combine_classes(
                    font_weight.bold,
                    font_size.lg,
                    m.b(4)
                )
            ),
            Div(
                Table(
                    Thead(
                        Tr(
                            Th("Plugin"),
                            Th("Execution"),
                            *[Th(label) for label in all_features.keys() if label != "Execution"]
                        )
                    ),
                    Tbody(
                        *[
                            Tr(
                                Td(
                                    plugin.title,
                                    cls=str(font_weight.bold) if plugin.get_unique_id() == selected_plugin_id else ""
                                ),
                                Td(format_execution_mode(plugin.execution_mode)),
                                *[
                                    Td(
                                        next(
                                            (val for pid, val in all_features[label] if pid == plugin.get_unique_id()),
                                            "N/A"
                                        )
                                    )
                                    for label in all_features.keys()
                                    if label != "Execution"
                                ],
                                cls=str(bg_dui.primary.opacity(10)) if plugin.get_unique_id() == selected_plugin_id else ""
                            )
                            for plugin in plugins
                        ]
                    ),
                    cls=combine_classes(table, table_modifiers.zebra, w.full)
                ),
                cls="overflow-x-auto"
            ),
            cls=combine_classes(
                card,
                card_body,
                bg_dui.base_100,
                m.t(4)
            )
        )
    )

## Main Selector Function

Factory function that routes to different strategies.

In [ ]:
#| export
def render_plugin_selector(
    plugins: List[PluginMetadata],  # Available plugins
    selected_plugin_id: Optional[str] = None,  # Currently selected plugin ID
    selection_endpoint: str = None,  # HTMX endpoint for selection
    target_id: str = None,  # HTMX target element ID
    strategy: str = "dropdown",  # UI strategy: "dropdown" or "comparison"
    get_plugin_config: Optional[Callable[[str], Dict[str, Any]]] = None,  # Function to get plugin config
    **kwargs  # Additional strategy-specific options
) -> FT:  # Plugin selector UI
    """Render plugin selector with the specified UI strategy.
    
    Strategies:
    - "dropdown": Dropdown selector with details panel (default)
    - "comparison": Comparison table of all plugins
    """
    
    if strategy == "comparison":
        return render_comparison_table(
            plugins=plugins,
            selected_plugin_id=selected_plugin_id,
            get_plugin_config=get_plugin_config
        )
    
    # Default to dropdown strategy
    return render_dropdown_selector(
        plugins=plugins,
        selected_plugin_id=selected_plugin_id,
        selection_endpoint=selection_endpoint,
        target_id=target_id,
        get_plugin_config=get_plugin_config,
        **kwargs
    )

## Examples

In [ ]:
# Example: Create sample plugins
from cjm_fasthtml_plugins.core.execution_mode import PluginExecutionMode

sample_plugins = [
    PluginMetadata(
        name="whisper_tiny",
        category="transcription",
        title="Whisper Tiny",
        description="Fast, lightweight transcription model",
        config_schema={"type": "object", "properties": {"device": {"type": "string"}}},
        version="1.0.0",
        is_configured=True,
        execution_mode=PluginExecutionMode.IN_PROCESS
    ),
    PluginMetadata(
        name="whisper_large",
        category="transcription",
        title="Whisper Large",
        description="High-quality transcription model",
        config_schema={"type": "object", "properties": {"device": {"type": "string"}}},
        version="2.0.0",
        is_configured=True,
        execution_mode=PluginExecutionMode.SUBPROCESS
    )
]

# Mock config getter
def mock_get_config(plugin_id: str) -> Dict[str, Any]:
    return {"device": "cuda", "supports_streaming": True}

# Render dropdown selector
selector = render_plugin_selector(
    plugins=sample_plugins,
    selected_plugin_id="transcription_whisper_tiny",
    selection_endpoint="/transcription/select",
    target_id="transcription-content",
    get_plugin_config=mock_get_config
)

selector

```html
<div id="plugin-selector">
  <div class="mb-4">
<label class="font-semibold mb-2">Select Plugin</label><select name="plugin_id" id="plugin-select" class="select w-full"><option value="transcription_whisper_tiny" selected>Whisper Tiny</option><option value="transcription_whisper_large">Whisper Large</option></select>  </div>
  <div id="plugin-details" class="card bg-base-200 mt-4">
    <div class="card-body">
      <h3 class="card-title">Whisper Tiny</h3>
      <p class="text-base-content/80 text-sm mb-4">Fast, lightweight transcription model</p>
      <div class="mb-4">
        <div class="flex items-center mb-2">
<span class="font-medium text-base-content/70 text-sm">Execution:</span><span class="badge badge-success badge-sm ml-2">Local (In-Process)</span>        </div>
        <div class="flex items-center mb-2">
<span class="font-medium text-base-content/70 text-sm">Device:</span><span class="badge badge-primary badge-sm ml-2">CUDA</span>        </div>
      </div>
      <p class="text-base-content/60 text-xs">Version: 1.0.0</p>
    </div>
  </div>
  <div class="mt-4">
<button hx-post="/transcription/select" hx-swap="innerHTML" hx-include="#plugin-select" hx-target="#transcription-content" class="btn btn-primary btn-md mt-4">Continue with Selected Plugin</button>  </div>
  <div class="mt-2">
<button hx-get="/transcription/compare" hx-swap="innerHTML" hx-target="#plugin-comparison" class="btn btn-outline btn-sm mt-2">Compare All Plugins</button>  </div>
  <div id="plugin-comparison" class="mt-4"></div>
</div>

```

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()